In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import polars as pl

import jacques
from jacques import kcqe
from data_pipeline.loader import FluDataLoader

## Loading the Flu Data

In [2]:
# Load data
fdl = FluDataLoader('data-raw')

combined_dat = fdl.load_data(hhs_kwargs={'rates': True})

/Users/bwrogers/Documents/Research/jacques_flu/.venv/lib/python3.12/site-packages/data_pipeline/loader.py:187: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[fill_cols] = df[fill_cols].fillna(axis=0, method='ffill')
/Users/bwrogers/Documents/Research/jacques_flu/.venv/lib/python3.12/site-packages/data_pipeline/loader.py:187: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[fill_cols] = df[fill_cols].fillna(axis=0, method='ffill')
/Users/bwrogers/Documents/Research/jacques_flu/.venv/lib/python3.12/site-packages/data_pipeline/loader.py:187: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[fill_cols] = df[fill_cols].fillna(axis=0, method='ffill')
/Users/bwrogers/Documents/Research/jacques_flu/.venv/lib/python3.12/si

In [3]:
state_dat = (pl.DataFrame(combined_dat).
             filter((pl.col("agg_level") == "state")).
             select(['location', 'source', 'wk_end_date', 'inc_trans_cs'])).to_pandas()

In [9]:
state_dat.head()

location,source,wk_end_date,inc_trans_cs
str,str,datetime[ns],f64
"""06""","""flusurvnet""",2010-10-09 00:00:00,-0.178518
"""06""","""flusurvnet""",2010-10-16 00:00:00,-0.178518
"""06""","""flusurvnet""",2010-10-23 00:00:00,-0.34235
"""06""","""flusurvnet""",2010-10-30 00:00:00,-0.178518
"""06""","""flusurvnet""",2010-11-06 00:00:00,-0.34235


## Featurizing the Data

We make use of the `timeseriesutils` module to featurize the data